# New Additions in Python 3.14
- Annotationlib
    - PEP 649
    - PEP 749
    - DO: Format, ForwardRef, get_annotations, annotations_to_string, get_annotate_function, value_to_string
    - DONT: call_annotate_function (reminds me of eval), call_evaluate_function (same), 

- Tools (3.13-3.14) (When Needed)
    - DO: assert_type, assert_never, reveal_type, final, override, get_type_hints, get_origin, get_args, is_typeddict
    - DONT: cast, overload, get_overloads, clear_overloads

- PEP 747

- Inlined Dictionaries

## Python 3.14 Annotations
### PEP 649: Deferred Evaluation of Annotations Using Descriptors
#### Purpose
- Help solve decoupled type hints

- Solve problems with forward-references and circular-references
    - PEP 563 tried to fix this through stringized annotations, but this caused problems at runtime

- This PEP's approach: lazily apply a new object method call \__annotate__
    - If it exists at compile time: the annotations dict is returned
    - \__annotations__ is a "data descriptor": once called, the result is cached
        - Provides three actions: get, set, and delete
        - Always defined, and may only be set to either None or to a callable
    - The callable stored in \__annotate__ must accept a single required positional argument called format, which is always an int (or a subclass of int)
        - Returns a dict or raises an error (NotImplementedError)

- \__annotate__(format: int) -> dict
    - Can only raise NameError for format=1; else NotImplementedError

- The get_annotations (annotationlib) is accessed via a keyword-only parameter: format

- Supported annotation on three different types: functions, classes, and modules

- Deleting the \__annotations__ attribute directly will also clear \__annotate__

- Gist: \__annotate__ now delays the evaluation of annotations until \__annotations__ is referenced in the future

In [ ]:
# Pseudocode
def get_annotations(o, format):
    if format == VALUE:
        return dict(o.__annotations__)

    if format == FORWARDREF:
        try:
            return dict(o.__annotations__)
        except NameError:
            pass

    if not hasattr(o.__annotate__):
        return {}

    c_a = o.__annotate__
    try:
        return c_a(format)
    except NotImplementedError:
        if not can_be_called_with_fake_globals(c_a):
            return {}
        c_a_with_fake_globals = make_fake_globals_version(c_a, format)
        return c_a_with_fake_globals(VALUE)

### PEP 749: Implementing PEP 649
#### Purpose
- Provide tweaks and additions to PEP 649
- Overview of annotationlib

### Annotationlib
#### Definitions
- Stock semantics (3.0-3.13): eager annotation evaluation
- Stringified (3.7+)
- Deferred evaluation (3.14+): lazy evaluation 

#### Format
- Format is an IntEnum: can be passed to get_annotations() and to \__annotate__ functions

- Format identifiers are always predefined integer values
    - VALUE = 1
        - Result of evaluating the annotation expressions
    - FORWARDREF = 2
        - Values are real annotation values (as per Format.VALUE format) for defined values, and ForwardRef proxies for undefined values. Real objects may contain references to, ForwardRef proxy objects.
        - If it encounters an undefined name/free variable thats not associated with a value, it dynamically creates a proxy object (a ForwardRef) that substitutes for that value in the expression
            - If all real values are defined at the time the function is called, inspect.FORWARDREF and inspect.VALUE produce identical results.
    - STRING = 3
        - stringized annotations
    - VALUE_WITH_FAKE_GLOBALS = 4
        - This format is only used internally, and shouldnt be passed

- FORWARDREF returns ForwardRef objects for annotations that cannot be resolved, allowing you to inspect the annotations without evaluating them.

#### ForwardRef
- A proxy object for forward references in annotations

- Returned when FORWARDREF format is used and annotations can't be resolved

- Methods
    1. \__forward_arg__: A string containing the code that was evaluated to produce the ForwardRef. The string may not be exactly equivalent to the original source.

    2. evaluate(*, globals=None, locals=None, type_params=None, owner=None)
        - Evaluates and returns its value
        - (IMPORTANT) This is where the symtable can be better utilized

- Once a ForwardRef instance has been evaluated, it caches the evaluated value, and future calls to evaluate() will return the cached value, regardless of the parameters passed in.

#### Retrieving Annotations
- get_annotations() is the main entry point

1. get_annotate_function(obj)
    - Retrieve the annotate function for obj, or None
    - Equivalent to, but better than, \__annotate__

2. get_annotations(obj, *, globals=None, locals=None, eval_str=False, format=Format.VALUE)
    - Compute the annotations dict for an object

In [ ]:
# Need to check all of these

In [ ]:
mytype = str
def foo(a:mytype): pass
mytype = int

print(foo.__annotations__['a'])
print(get_annotations(foo)['a'])
## Will print -> <class 'int'>

In [ ]:
def func(a: Cls) -> None:
    print(a)

class Cls: pass

print(func.__annotations__)

In [ ]:
from copy import deepcopy

annotations = {'x': int, 'y': MyType, 'return': float}

def annotate_foo():
    return deepcopy(annotations)

def foo(x = 3, y = "abc"):
    ...

# Check that it's not __annotate__
foo.__annotations = deepcopy(annotations)
## OR ##
foo.__annotations__ = annotate_foo

class MyType:
    ...

# Check that it's not __annotate__
foo_y_annotation = foo.__annotations__['y']

In [ ]:
""" 
    Two Potential Ways For ForwardRef Assignment
        1. Check and ForwardRef.evaluate()
        2. Assign within scope before use
"""

from typing import Callable, Tuple, Concatenate, TypeGuard
from annotationlib import get_annotations, Format, ForwardRef

type Info = Tuple[str, ...]
type Generic_SignUp = Callable[Concatenate[int, Info, ...], bool]

def test_this(from_website: NewSignUp) -> TypeGuard[bool]:
    ...

# Try here!!!
for k, v in get_annotations(test_this):
    if isinstance(v, Format.VALUE):
        value_type = 'VALUE' 
    elif isinstance(v, ForwardRef) or isinstance(v, Format.ForwardRef):
        value_type = 'FORWARDREF' 
    else:
        value_type = 'ERROR'

    print(f'Name of arg is {k} and is of type {value_type}')

    if value_type == 'FORWARDREF':
        print('Lets fix that issue here!')

        # Way One
        ## Check
        test_this.__annotations__[k].evaluate(Generic_SignUp)

# Way Two
# UUID, Info -> bool
type NewSignUp = Callable[Concatenate[int, Info, ...], bool]

# Try here!!!
for k, v in get_annotations(test_this):
    if isinstance(v, Format.VALUE):
        value_type = 'VALUE' 
    elif isinstance(v, ForwardRef) or isinstance(v, Format.ForwardRef):
        value_type = 'FORWARDREF' 
    else:
        value_type = 'ERROR'

    # Should be of type Format.VALUE -> by either way!!!
    print(f'Name of arg is {k} and is of type {value_type}')

### Inlined Typed Dictionaries (PEP 764)
- Scope-aware, closed type dictionaries
    - No extra items PEP
    - Generics work for the scope provided
    - Total inclusion by design, Required type qualifier by default (not needed)
    - Anonymous-style TypedDict declarations

In [ ]:
class C[T]:
    inlined_td: TypedDict[{'name': T}]  # OK, `T` is scoped to the class `C`.

reveal_type(C[int]().inlined_td['name'])  # Revealed type is 'int'


def fn[T](arg: T) -> TypedDict[{'name': T}]: ...  # OK: `T` is scoped to the function `fn`.

reveal_type(fn('a')['name'])  # Revealed type is 'str'


type InlinedTD[T] = TypedDict[{'name': T}]  # OK, `T` is scoped to the type alias.


T = TypeVar('T')

InlinedTD = TypedDict[{'name': T}]  # OK, same as the previous type alias, but using the old-style syntax.


def func():
    InlinedTD = TypedDict[{'name': T}]  # Not OK: `T` refers to a type variable that is not bound to the scope of `func`.

In [ ]:
# Old Way
from supporting.vultures.rsa_crypto import RSA_Crypto
from typing import TypedDict, ReadOnly, Required, NotRequired, Unpack, Any, TypeGuard, Optional
from uuid import UUID, uuid4
from datetime import datetime

type Date = datetime

class PublicCipher(TypedDict):
    public_key: ReadOnly[Any]
    private_key: ReadOnly[Any]
    cipher_type: str
    created: ReadOnly[Date]
    
class RsaObj(TypedDict):
    obj: ReadOnly[RSA_Crypto]
    created: ReadOnly[Date]
    
class KeyRing(TypedDict, total=False):
    account_id: ReadOnly[UUID]
    encryption_keys: Required[PublicCipher | RsaObj]
    updated: Required[Date]
    
def create_keyring[T: (KeyRing)]() -> T:
    created = datetime.now()
    return KeyRing(uuid4(), RsaObj(RSA_Crypto(), created), created)
    
def check_keyring_encryption[T: (KeyRing)](**kwargs: T) -> TypeGuard[RsaObj]:
    return isinstance(kwargs['encryption_keys'], RsaObj)
    
def update_keyring[T: (KeyRing)](**kwargs: T) -> T:
    if check_keyring_encryption(**kwargs):
        kwargs['encryption_keys'] = RsaObj(RSA_Crypto(), datetime.now())
    else:
        # Implement other crypto assignment to the PublicCipher class
        pass
    
    return kwargs

# New Way... Check in October
type KeyPair[T] = TypedDict[{'public_key': ReadOnly[T], 'private_key': ReadOnly[T]}]
type PublicKeysObj[S: str, T, D: (Date)] = TypedDict[
                                        {
                                            'obj': NotRequired[ReadOnly[KeyPair[T] | RSA_Crypto[T]]],
                                            'cipher_type': S,
                                            'created': D
                                        }
                                    ]
type KeyRing[E: (PublicCipher, RsaObj)] = TypedDict[{'account_id': ReadOnly[UUID], 
                                                    'encryption_keys': E, 'updated': Date}]

def create_keyring() -> KeyRing:
    ...

### Annotating Type Forms (PEP 747)
- typing.TypeForm: helps provide a type form object at runtime, which encodes the info supplied in the type expression
    - First way to accept type validated functional parameters that have certain type form objects, and/or, the ability to work with such parameter

- Reread in the future to see if I missed anything

In [ ]:
from typing import Any, Callable, TypeForm, ClassVar, Protocol, Tuple
from atomic_varible import AtomicVariable
from supporting.utils import Controlled_UUID
from uuid import UUID 

class Lockable(Protocol):
    ...

type Wrapped_Callable[**P, T] = Tuple[T, Callable[P, Any]]

# Custom_RCU wraps calls to log_interceptor with a lockable unit
## Could be Rust-based atomics: CAS
class Collector[**P, T: UUID, Controller](Meta=Custom_RCU):
    controlled_ids: ClassVar[Controller] = Controlled_UUID()

    @classmethod
    def log_interceptor(cls: Lockable, f: Callable[P, Any]) -> TypeForm[Wrapped_Callable]:
        new_uuid = cls.controlled_ids.get_uuid()

        # Run the callable in the wrapped RCU metaclass
        return f 